In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
from multiprocessing import Pool

In [2]:
train=pd.read_csv('../train.csv')
test=pd.read_csv('../test_QyjYwdj.csv')
item=pd.read_csv('../item_data.csv')
transaction=pd.read_csv('../customer_transaction_data.csv')
coupon=pd.read_csv('../coupon_item_mapping.csv')
camp=pd.read_csv('../campaign_data.csv')
camp.start_date=pd.to_datetime(camp.start_date,format='%d/%m/%y')
camp.end_date=pd.to_datetime(camp.end_date,format='%d/%m/%y')
camp.sort_values('start_date',inplace=True)
df=pd.concat((train.iloc[:,:-1],test),axis=0)
df=df.merge(camp,on='campaign_id',how='left')
transaction.date=pd.to_datetime(transaction.date)

In [3]:
df.head()

,id,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date
0,1,13,27,1053,X,2013-05-19,2013-07-05
1,2,13,116,48,X,2013-05-19,2013-07-05
2,6,9,635,205,Y,2013-03-11,2013-04-12
3,7,13,644,1050,X,2013-05-19,2013-07-05
4,9,8,1017,1489,X,2013-02-16,2013-04-05


In [4]:
coupon_list=df.coupon_id.unique()

In [5]:
def get_similar_coupon_mean(c):
    coups=coupon[coupon.item_id.isin(coupon[coupon.coupon_id==c].item_id)].coupon_id.unique()
    coups=coups[np.where(coups!=c)[0]]
    return c,train[train.coupon_id.isin(coups)].redemption_status.sum(),train[train.coupon_id.isin(coups)].redemption_status.mean()

In [6]:
%%time
cols=['coupon_id','tot_redemption','mean_redemption']
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_similar_coupon_mean, [coup for coup in coupon_list])
finally: 
    pool.close()
    pool.join()

CPU times: user 24 ms, sys: 64 ms, total: 88 ms
Wall time: 694 ms


In [7]:
pd.DataFrame(data_outputs,columns=cols).to_csv('../coupon_mean_encoding_v2.csv',index=False)

In [8]:
transaction=transaction.merge(item,on='item_id')

In [9]:
coupon=coupon.merge(item,on='item_id')

In [10]:
coupon.head()

,coupon_id,item_id,brand,brand_type,category
0,105,37,56,Local,Grocery
1,6,37,56,Local,Grocery
2,22,37,56,Local,Grocery
3,31,37,56,Local,Grocery
4,107,75,56,Local,Grocery


In [11]:
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id==data_['customer_id'])&\
                         (transaction.brand.isin(coupon[coupon.coupon_id==data_['coupon_id']].brand))\
               &(transaction.date<data_['start_date'])]
    
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_.id]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_.id]+np.repeat(0,11).tolist()
    return feats

In [12]:
import datetime
temp=df[['customer_id','coupon_id','start_date','id','end_date']]
temp['cut_off_date']=pd.to_datetime(df.start_date) -datetime.timedelta(150)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
%%time
cols=['id','num_bought_brand_cust','num_discount_availed_brand_cust','frac_discount_availed_brand_cust','sum_discount_brand_cust',
'selling_price_sum_brand_cust',
'coupon_discount_mean_brand_cust','selling_price_mean_brand_cust','other_discount_sum_brand_cust',
      'other_discount_mean_brand_cust','cd_sp_ratio_mean_brand_cust','cd_sp_ratio_sum_brand_cust'
     ]

try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()

CPU times: user 20.2 s, sys: 572 ms, total: 20.8 s
Wall time: 12min 12s


In [14]:
feat_df=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

In [15]:
feat_df.to_csv('../brand_cust_feats_no_lag_v2.csv',index=False)

In [16]:
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.brand.isin(coupon[coupon.coupon_id==data_['coupon_id']].brand))\
               &(transaction.date<data_['start_date'])]

    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_.id]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_.id]+np.repeat(0,11).tolist()
    return feats

In [17]:
%%time
# cols=['id','num_bought_brand','num_discount_availed_brand','frac_discount_availed_brand','sum_discount_brand']
cols=['id','num_bought_brand','num_discount_availed_brand','frac_discount_availed_brand','sum_discount_brand',
'selling_price_sum_brand',
'coupon_discount_mean_brand','selling_price_mean_brand','other_discount_sum_brand',
      'other_discount_mean_brand','cd_sp_ratio_mean_brand','cd_sp_ratio_sum_brand'
     ]
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()

CPU times: user 20 s, sys: 772 ms, total: 20.7 s
Wall time: 20min 50s


In [18]:
feat_df=pd.DataFrame(np.row_stack(data_outputs),columns=cols)
feat_df.to_csv('../brand_feats_no_lag_v2.csv',index=False)

In [21]:
# a=pd.read_csv('../brand_cust_feats_no_lag_v2.csv')
# b=pd.read_csv('../../av_amex/brand_cust_feats_no_lag_v2.csv')
# (a.mean()==b.mean()).value_counts()